In [19]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import json

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()



prompt_name = 'confidence'
prompt = p.confidence
MODEL = "gpt-3.5-turbo"
TEMP = 0

file = "dynamic_fewshot_multi_table_gpt-4-turbo-preview"

DATA_PATH = f'../../Datasets/Evaluations/Schema Matching/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Schema Matching/Error_Analysis/Confidence Analysis/{MODEL}_{prompt_name}_{file}.csv"






In [20]:
df = pd.read_csv(DATA_PATH)
df['confidence'] = None



In [21]:
def execute(df, prompt, start = 0):
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=100)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=100)
    for i in range(len(df)):
        if(df["confidence"][i] != None and df["confidence"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        user_prompt = df.loc[i, 'prompt']
        ai_answer = str(df.loc[i, 'ai_answer'])
        match = ""
        if str(df.loc[i, 'y_pred']).lower() == 'false':
            match = "not match"
        else:
            match = "match"
        columns = "Column A-" + str(df.loc[i, 'column_index_left']) + " and Column B-" + str(df.loc[i, 'column_index_right'])
        try: 
            result = chain.run(user_prompt = user_prompt, ai_answer = ai_answer, match = match, columns = columns, callbacks=[handler])
        except Exception as e:
            print(e)
            print(chain.prompt.format_prompt(user_prompt = user_prompt, ai_answer = ai_answer, match = match, columns = columns).to_string())
            df.loc[i, 'confidence'] = 'error'
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(user_prompt = user_prompt, ai_answer = ai_answer, match = match, columns = columns).to_string()
        if (i < 50):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        df.loc[i, 'confidence']= result
    return df


df = execute(df, prompt)

    



0
System: You are a helpful AI.
Human: Human: Example 1: 
Input:
Table A: 
| Column A-0      |   Column A-1 | Column A-2     |   Column A-3 | Column A-4                   | Column A-5   | Column A-6   |
|:----------------|-------------:|:---------------|-------------:|:-----------------------------|:-------------|:-------------|
| Oceania         |          nan | nan            |       nan    | nan                          | nan          | nan          |
| Suriname        |           89 | 88;92.3,84.1   |        69.01 | 48Chr,27.4Hin,19.6Mus,5Sat   | $7,100       | 439,117      |
| Central America |          nan | nan            |       nan    | nan                          | nan          | nan          |
| Romania         |           94 | 98.4;99.1,97.7 |        71.63 | 86.8Ort,7.5Prot,4.7Cath,1Oth | $8,800       | 22,303,552   |
| San Marino      |          nan | 96;97,95       |        81.71 | Cath                         | $34,100      | 29,251       |

Table B: 
| Column B-0      

In [22]:
print(df['confidence'])

0      Confidence: 100%
1       Confidence: 90%
2      Confidence: 100%
3      Confidence: 100%
4       Confidence: 80%
             ...       
445    Confidence: 100%
446    Confidence: 100%
447    Confidence: 100%
448    Confidence: 100%
449     Confidence: 90%
Name: confidence, Length: 450, dtype: object


In [18]:
df.to_csv(OUTPUT_PATH, index=False)
